## Oil Prices
##### Precio Petróleo
Precio Petróleo Crudo Ecuatoriano: Corresponde al precio promedio ponderado mensual de las exportaciones de los crudos Oriente y Napo realizadas por EP Petroecuador, sin que signifique que los crudos Oriente y Napo se mezclen para su exportación, la exportación se realiza por separado.

##### Inflación
**Inflación mensual**: Indicador económico que mide la evolución mensual de los precios de bienes y servicios que consumen los hogares.

**Inflación anual**: Indicador económico que mide la evolución del mes en referencia respecto al mismo mes del año anterior, de los precios de bienes y servicios que consumen los hogares. Es decir, la variación de los últimos doce meses.

**Inflación acumulada**: Indicador económico que mide la evolución mensual de los precios de bienes y servicios que consumen los hogares, respecto a diciembre del año anterior.

In [ ]:
import time
import json
from pathlib import Path
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options


urls = ['https://contenido.bce.fin.ec/documentos/informacioneconomica/indicadores/real/PrecioPetroleoCrudoEcuatoriano.html',
        'https://contenido.bce.fin.ec/documentos/informacioneconomica/indicadores/real/Inflacion.html']
opts = Options()
opts.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36")
data = []

driver = webdriver.Chrome(options=opts)
driver.get(url)
time.sleep(2)
table = driver.find_element(By.TAG_NAME, 'tbody')
rows = table.find_elements(By.TAG_NAME, 'tr')

data_dir = Path('data')
if (data_dir / 'oilPrice.csv').exists():
    df_prices = pd.read_csv(data_dir / 'oilPrice.csv')
    last_saved_period = df_prices.iloc[0]
    last_period = rows[0].text.split()
    if last_saved_period[0] == last_period[0] and last_saved_period[1] == last_period[1]:
        print('There is no new prices')
    else:
        for row in rows:
            data.append(row.text.split())
        df = pd.DataFrame(data,columns=['Year','Month','Price (USD)'])
        df['Price (USD)'] = df['Price (USD)'].str.replace(',','.').astype(float)
        df.to_csv(data_dir / 'oilPrice.csv')

for row in rows:
    data.append(row.text.split())
df = pd.DataFrame(data,columns=['Year','Month','Price (USD)'])
df['Price (USD)'] = df['Price (USD)'].str.replace(',','.').astype(float)
df.to_csv(data_dir / 'oilPrice.csv')

driver.quit()


KeyboardInterrupt: 

In [9]:
import os
os.chdir('..')

In [53]:
%pwd

'c:\\Users\\User\\Desktop\\DS\\Projects\\oil-optimization'

In [23]:
import time
from pathlib import Path
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options

def scrape_table_rows(driver):
    time.sleep(3)

    year_button = driver.find_element(By.XPATH, "//select[@id='añoInicio']")
    year_button.click()

    start_year = year_button.find_element(By.XPATH, ".//option[@value='2015']")
    time.sleep(2)
    start_year.click()

    time.sleep(2)
    year_button.click()

    table_head = driver.find_element(By.TAG_NAME, 'thead')
    content = table_head.find_elements(By.TAG_NAME, 'th')

    table_body = driver.find_element(By.TAG_NAME, 'tbody')
    rows = table_body.find_elements(By.TAG_NAME, 'tr')

    columns = [cont.text for cont in content]
    data = np.array([row.text.split() for row in rows])
    for i in np.arange(2,data.shape[1]):
         data[:,2] = [x.replace(',','.') for x in data[:,2]]

    df = pd.DataFrame(data,columns=columns)
    cols = df.columns[2:]
    df[cols] = df[cols].astype(float)

    return df

def update_csv(file_path, new_df):
    if file_path.exists():
        df_old = pd.read_csv(file_path)
        last_saved_year, last_saved_month = df_old.iloc[0,:2]
        last_new_year, last_new_month = new_df.iloc[0,:2]

        if (last_saved_year == last_new_year) and (last_saved_month == last_new_month):
            print('There are no new prices.')
            return


    new_df.to_csv(file_path,index=False)
    print('Data Updated')

if __name__ == '__main__':
    urls = ['https://contenido.bce.fin.ec/documentos/informacioneconomica/indicadores/real/PrecioPetroleoCrudoEcuatoriano.html',
        'https://contenido.bce.fin.ec/documentos/informacioneconomica/indicadores/real/Inflacion.html']
    data_dir = Path('data')
    data_dir.mkdir(exist_ok=True)

    opts = Options()
    opts.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36")
    opts.add_argument("--headless=new")

    for url in urls:
        driver = webdriver.Chrome(options=opts)
        file_name = url.split('/')[-1].split('.')[0]
        file_path = data_dir / f'{file_name}.csv'
        driver.get(url)
        time.sleep(2)

        df = scrape_table_rows(driver)
        update_csv(file_path, df)
        driver.quit()


Data Updated
Data Updated


In [ ]:
urls = ['https://contenido.bce.fin.ec/documentos/informacioneconomica/indicadores/real/PrecioPetroleoCrudoEcuatoriano.html',
        'https://contenido.bce.fin.ec/documentos/informacioneconomica/indicadores/real/Inflacion.html']
opts = Options()
opts.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36")
# opts.add_argument("--headless=new")
driver = webdriver.Chrome(options=opts)
driver.get(urls[0])
time.sleep(2)

table_head = driver.find_element(By.TAG_NAME, 'thead')
content = table_head.find_elements(By.TAG_NAME, 'th')

table_body = driver.find_element(By.TAG_NAME, 'tbody')
rows = table_body.find_elements(By.TAG_NAME, 'tr')

columns = [cont.text for cont in content]
data = np.array([row.text.split() for row in rows])

for i in np.arange(2,data.shape[1]):
     data[:,2] = [x.replace(',','.') for x in data[:,2]]


df = pd.DataFrame(data,columns=columns)
cols = df.columns[2:]
df[cols] = df[cols].astype(float)

driver.quit()

In [ ]:
df = pd.DataFrame(data,columns=['Year','Month','Price (USD)'])
df['Price (USD)'] = df['Price (USD)'].str.replace(',','.').astype(float)
df['Year'] = df['Year'].astype(int)

## Oil Indexes
Features:
* PRODUCCIÓN DE PETRÓLEO CRUDO (Miles de barriles)
* Exportaciones
* Volumen de transporte del Sistema de Oleoducto Transecuatoriano (SOTE) y Oleducto de Crudos Pesados (OCP)

In [35]:
import requests
from bs4 import BeautifulSoup
url1 = 'https://contenido.bce.fin.ec/documentos/informacioneconomica/SectorReal/ix_PetroleoMinas.html'

r = requests.get(url1, verify='bce.crt')
soup = BeautifulSoup(r.text)

In [36]:
soup.find(name='section',attrs={'id':'section_CSPxls'}).find_all(name='a')[1].get('href')

'https://contenido.bce.fin.ec/documentos/Estadisticas/Hidrocarburos/SerieCifrasPetroleras.xlsx'

In [64]:
import requests
import pandas as pd
import numpy as np

# r = requests.get('https://contenido.bce.fin.ec/documentos/Estadisticas/Hidrocarburos/SerieCifrasPetroleras.xlsx', verify='bce.crt')
# with open('/data/oilFigures.xlsx', 'wb') as f:
#     f.write(r.content)

df = pd.read_excel('data/oilFigures.xlsx',nrows=161)
years = df.iloc[0].unique()[2:].astype(int)

df = df.iloc[:,1:]
idx = df[df.iloc[:,0] == 'PRODUCCIÓN DE PETRÓLEO CRUDO (Miles de barriles)'].index[0]
df = df.iloc[idx+1,1:]


data = pd.DataFrame()
for i,year in enumerate(years):
    chunk = df[i*12:i*12+12].to_frame()
    chunk['year'] = year
    data = pd.concat([data,chunk],axis=0)


data.reset_index(drop=True).rename(columns={3:'Total'})

,Total,year
0,16029.77266,2007
1,14202.41541,2007
2,14944.46022,2007
3,15061.70616,2007
4,15862.39327,2007
...,...,...
218,13996.89146,2025
219,14148.01696,2025
220,14461.57249,2025
221,14014.07711,2025


In [2]:
import pandas as pd

In [28]:
df = pd.read_excel('data/oilFigures.xlsx',nrows=161)
years = df.iloc[0].unique()[2:].astype(int)
months = df.iloc[1,2:]

df = df.iloc[:,1:]
idx = df[df.iloc[:,0].isin(['PRODUCCIÓN DE PETRÓLEO CRUDO (Miles de barriles)',
                            'EXPORTACIONES DE PETRÓLEO CRUDO (Miles de barriles)',
                            ' - Transporte Promedio Diario',
                            'SOTE (11)'])].index

df = df.iloc[idx+1,1:].T

data = pd.DataFrame()
for i,year in enumerate(years):
    chunk = df[i*12:i*12+12].copy()
    chunk['year'] = year
    data = pd.concat([data,chunk],axis=0).reset_index(drop=True)
data['month'] = months.values

In [31]:
oil_columns = ['PRODUCCIÓN DE PETRÓLEO CRUDO (Miles de barriles)',
               'EXPORTACIONES DE PETRÓLEO CRUDO (Miles de barriles)',
               'SOTE (11)',
               'OCP (12)',
               'year',
               'month']

data.columns = oil_columns

In [ ]:
data.to_csv('data/oilLogistics.csv',index=False)

## Energy Information Administration (EIA) Api
Features:
* WTI
* Brent
* US oil exports to Ecuador

In [3]:
import json
import pandas as pd
import requests
from dotenv import dotenv_values

secrets = dotenv_values('../.env')

url = 'https://api.eia.gov/v2/crude-oil-imports/data'
url1 = 'https://api.eia.gov/v2/petroleum/pri/spt/data'


payload_imports = {
    'api_key':secrets['API_KEY'],
    'facets[originId][]':'CTY_EC',
    'data[]':'quantity',
    'start':'2015-01-01'
}

payload_wti = {
    'api_key':secrets['API_KEY'],
    'facets[product][]':'EPCWTI',
    'start':'2015-01-01',
    'sort[0][column]':'period',
    'sort[0][direction]':'asc',
    'frequency':'daily'
}

payload_brent = {
    'api_key':secrets['API_KEY'],
    'facets[product][]':'EPCBRENT',
    'start':'2015-01-01',
    'sort[0][column]':'period',
    'sort[0][direction]':'asc',
    'frequency':'daily'
}

r = requests.get(url=url, params=payload_imports)
r1 = requests.get(url=url1,params=payload_wti)
r2 = requests.get(url=url1,params=payload_brent)

imports_df = pd.DataFrame(r.json()['response']['data'])
wti_df = pd.DataFrame(r1.json()['response']['data'])
brent_df = pd.DataFrame(r2.json()['response']['data'])


imports_df.to_csv('data/imports.csv', index=False)
wti_df.to_csv('data/wtiPrice.csv', index=False)
brent_df.to_csv('data/brentPrice.csv', index=False)

tym2loT7krvfyUGqx5vJSgN2ciqVizgsD8i4SUIC


## Stock Market Indexes
* S&P 500 Energy

In [56]:
import yfinance as yf

In [ ]:
energy = yf.download('^GSPE',start='2015-01-01')    

C:\Users\User\AppData\Local\Temp\ipykernel_1752\627084361.py:1: FutureWarning: YF.download() has changed argument auto_adjust default to True
  energy = yf.download('^GSPE',start='2015-01-01')
[*********************100%***********************]  1 of 1 completed


In [78]:
energy.Close.reset_index().to_csv('data/energy.csv',index=False)

In [13]:
import pprint
from src.oil_optimization.utils.io_helpers import read_yaml
content = read_yaml('config/config.yml')
pprint.pprint(content)

{'data_ingestion': {'api_endpoint': ['https://api.eia.gov/v2/crude-oil-imports/data',
                                     'https://api.eia.gov/v2/petroleum/pri/spt/data'],
                    'macro_indicators_urls': ['https://contenido.bce.fin.ec/documentos/informacioneconomica/indicadores/real/PrecioPetroleoCrudoEcuatoriano.html',
                                              'https://contenido.bce.fin.ec/documentos/informacioneconomica/indicadores/real/Inflacion.html'],
                    'oil_reports_url': ['https://contenido.bce.fin.ec/documentos/Estadisticas/Hidrocarburos/SerieCifrasPetroleras.xlsx'],
                    'root_dir': 'data',
                    'yfinance': {'start_date': '2015-01-01',
                                 'ticker': '^GSPE'}}}
